In [ ]:
# NECESSARY IMPORTS

import sys
sys.path.insert(0, 'modules/')
import mercury as mr
import ee
import geemap.foliumap as geemap
import os
import datetime
import webbrowser
import create_ee_creds as cec
import txtreading as txtr
from colorama import Fore

In [ ]:
# INITIALIZING INTERFACE ELEMENTS

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎**')
mr.Markdown(text = '**🌎 This program solution is not final and some details are potentially to be changed 🌎**')
mr.Markdown(text = '**🌏 Current version - 0.31b 🌏**')

text = 'OSINT-TECHNOLOGIES'
text_params = {
    'fontsize': 10,
    'fontcolor': 'black',
    'bold': True,
    'padding': '3px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '1px',
    'position': 'bottomleft',
}

sources_list = ["Google EE"]
control_panel_choice = mr.Select(value="Google EE", choices=sources_list, label="Choose the source control panel")

app  = mr.App(title="GAIA v0.31b [BETA] 🌐",
             description="Open-Source Geospatial & Aerial Images Analyser",
             show_code=False,
             show_prompt=False,
             continuous_update=True,
             static_notebook=False,
             show_sidebar=True,
             full_screen=True,
             allow_download=False)

if control_panel_choice.value == "Google EE":
    ee_user_guide = mr.Button(label="How to start Google EE")
    ee_reg = mr.Button(label="EE registration")
    ee_service_acc = mr.Button(label="Create EE service account")
    save_map_html = mr.Button(label="Save current map [HTML]")
    ds_info = mr.Checkbox(value=False, label='Show datasets technical info')
    date_one = mr.Text(value="2022-01-01", label="Enter start date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter final date (format YYYY-MM-DD)", rows=1)
    max_cloud_covering = mr.Slider(value=80, label='Max cloud cover (%, may affect datasets quality)', min=0, max=100)
    datasets_landsat = mr.Checkbox(value=False, label="Show Landsat datasets")

    if datasets_landsat.value:
        ls7bands = mr.MultiSelect(label="Select LandSat 7 SR bands",
                                  value=['SR_B3', 'SR_B2', 'SR_B1'],
                                  choices=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_ATMOS_OPACITY', 'SR_CLOUD_QA'])

        ls8bands = mr.MultiSelect(label="Select LandSat 8 bands",
                                  value=['B4', 'B3', 'B2'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL'])

        ls9bands = mr.MultiSelect(label="Select LandSat 9 SR bands",
                                  value=['SR_B4', 'SR_B3', 'SR_B2'],
                                  choices=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL'])


        ls9rawbands = mr.MultiSelect(label="Select LandSat 9 RAW bands",
                                  value=['B4', 'B3', 'B2'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL'])


    datasets_sentinel = mr.Checkbox(value=False, label="Show Sentinel datasets")
    if datasets_sentinel.value:
        sen2msibands = mr.MultiSelect(label="Select Sentinel 2 MSI (TOA/SR) bands",
                                  value=['B2', 'B3', 'B4'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])


    datasets_modis = mr.Checkbox(value=False, label="Show MODIS datasets")
    datasets_other = mr.Checkbox(value=False, label='Show other datasets')

mr.Markdown(text = "#GOOGLE EARTH ENGINE MAP 🌐")
mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))

if ds_info.value:
    txtr.ds_info_read()

if ee_user_guide.clicked:
    txtr.ee_guide_read()

In [ ]:
# GOOGLE EE AUTHENTICATION AND INITIALIZATION

if ee_reg.clicked:
    webbrowser.open_new_tab('https://code.earthengine.google.com/register')

if ee_service_acc.clicked:
    webbrowser.open_new_tab('https://console.cloud.google.com/iam-admin/serviceaccounts/')

if str((cec.service_acc_filename).rstrip()) not in os.listdir():
    print(Fore.RED + '[File/directory missing error]. Your EE credentials .JSON file was not found in GAIA directory')
    mr.Stop()
else:
    key_file = (cec.service_acc_filename).rstrip()
    service_account = (cec.service_acc_mail).rstrip()
    credentials = ee.ServiceAccountCredentials(service_account, key_file)
    ee.Initialize(credentials)

In [ ]:
# MAP AND DATASETS SETTINGS

m = geemap.Map()
m.add_text(text, **text_params)


if datasets_landsat.value:

    #LANDSAT 7
    ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUD_COVER', max_cloud_covering.value))
    vis_ls7 = {'bands': ls7bands.value}

    #LANDSAT 8
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUD_COVER', max_cloud_covering.value))
    vis_ls8 = {
        'bands': ls8bands.value,
        'min': 0.0,
        'max': 0.4,
    }

    #LANDSAT 9
    def apply_scale_factors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

    collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUD_COVER', max_cloud_covering.value))
    median = collection.median()
    ls9 = apply_scale_factors(median)

    vis_ls9 = {
        'bands': ls9bands.value,
        'min': 0.0,
        'max': 0.3,
    }

    #LANDSAT 9 RAW
    ls9raw = ee.ImageCollection("LANDSAT/LC09/C02/T2").filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUD_COVER', max_cloud_covering.value))
    ls9raw_tc = ls9raw.select(ls9rawbands.value)
    vis_ls9raw_tc = {
        'min': 0.0,
        'max': 30000.0,
    }


if datasets_sentinel.value:
    #SENTINEL 2 MSI
    sen2msi_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_covering.value))
    composite_sr = sen2msi_sr.median()

    vis_sen2sr = {
        'bands': sen2msibands.value,
        'min': 0,
        'max': 3000,
        'gamma': 1.4,
    }

    sen2msi_toa = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(str(date_one.value), str(date_two.value)).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_covering.value))
    composite_toa = sen2msi_toa.median()

    vis_sen2toa = {
        'bands': sen2msibands.value,
        'min': 0,
        'max': 3000,
        'gamma': 1.4,
    }


    #SENTINEL 5P CLOUDS
    sen5pc = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").select('cloud_fraction').filterDate(str(date_one.value), str(date_two.value))
    vis_sen5pc = {
        min: 0,
        max: 0.95,
        'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    }


if datasets_modis.value:
    modis_sc = ee.ImageCollection('MODIS/061/MOD10A1').filter(ee.Filter.date('2018-01-01', '2018-05-01'))
    modis_snowcover = modis_sc.select('NDSI_Snow_Cover')
    vis_modis_sc = {
        min: 0.0,
        max: 100.0,
        'palette': ['black', '0dffff', '0524ff', 'ffffff'],
    }

    modis_tsr = ee.ImageCollection('MODIS/061/MOD09GQ').filterDate(str(date_one.value), str(date_two.value))
    vis_modis_tsr = {
        min: -100.0,
        max: 8000.0,
        'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01']
    }

    modis_tta = ee.ImageCollection('MODIS/061/MOD14A1').filterDate(str(date_one.value), str(date_two.value))
    vis_modis_tta = {
        min: 0.0,
        max: 6000.0,
        'bands': ['MaxFRP', 'FireMask', 'FireMask'],
    }




if datasets_other.value:
    firms = ee.ImageCollection("FIRMS").filterDate(str(date_one.value), str(date_two.value))
    vis_firms = {
        min: 325.0,
        max: 400.0,
    }

    alos_td = ee.Image("CSP/ERGo/1_0/Global/ALOS_topoDiversity")
    alos_topodiv = alos_td.select('constant')
    vis_alos_td = {
        min: 0.0,
        max: 1.0,
    }



In [ ]:
# MAPS AND DATASETS INITIALIZATION


m.add_basemap('SATELLITE')
m.add_basemap('TERRAIN')
m.add_basemap('HYBRID')

try:
    if datasets_landsat.value:
        m.addLayer(ls7, vis_ls7, "LandSat 7 SR", True, 0.7)
        m.addLayer(ls8, vis_ls8, 'LandSat 8 TOA')
        m.addLayer(ls9, vis_ls9, 'LandSat 9 SR')
        m.addLayer(ls9raw_tc, vis_ls9raw_tc, 'LandSat 9 RAW')


    if datasets_sentinel.value:
        m.addLayer(composite_sr, vis_sen2sr, 'Sentinel 2 MSI SR')  #SENTINEL 2 MSI
        m.addLayer(composite_toa, vis_sen2toa, 'Sentinel 2 MSI TOA')
        m.addLayer(sen5pc, vis_sen5pc, 'Sentinel 5P Cloud')  #SENTINTEL 5P CLOUD


    if datasets_modis.value:
        m.addLayer(modis_snowcover, vis_modis_sc, 'MODIS Snowcower')  #MODIS snowcower
        m.addLayer(modis_tsr, vis_modis_tsr, 'MODIS TSR')  #MODIS snowcower
        m.addLayer(modis_tta, vis_modis_tta, 'MODIS TTA&FD')


    if datasets_other.value:
            m.addLayer(firms, vis_firms, 'FIRMS')  #FIRMS
            m.addLayer(alos_topodiv, vis_alos_td, 'ALOS TD') #ALOS TD

except ee.EEException as error:
    print(Fore.RED + '[Map plotting/initialization error] - {}'.format(str(error)[17:]))
    pass

In [ ]:
# CALLING MAP OBJECT

m

In [ ]:
# POST-PROCESSING FUNCTIONS

download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
today = datetime.datetime.today()
time = today.strftime("%Y%m%d-%H.%M.%S")
timestr = str(time)

if save_map_html.clicked:
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    html_file = os.path.join(download_dir, 'GAIA-{}.html'.format(timestr))
    m.to_html(filename=html_file, title='My Map', width='100%', height='880px')